In [3]:
import os


from sollfahrplanreader.currentdayanalyser import GTFSCurrentDayAnalysis


myAnalyser=GTFSCurrentDayAnalysis(os.getcwd())
todaysServices=myAnalyser.getTodaysServices()
todaysServices

20240515


,service_id,date,exception_type
63,Special,20240515,1
543,Special#7,20240515,1
760,Special#9,20240515,1
928,Special#10,20240515,1
1172,Special#12,20240515,1
...,...,...,...
1005748,Special#16761,20240515,1
1005921,Special#16766,20240515,1
1005959,Special#16768,20240515,1
1006031,Special#16774,20240515,1


In [4]:
from datetime import datetime
from exceptions.noflatfile import NoFlatFileForTodayException

raise NoFlatFileForTodayException

NoFlatFileForTodayException: Database already filled for current day 2024-05-21 10:23:08

In [1]:

import unittest
myAnalyser=GTFSCurrentDayAnalysis(os.getcwd())
from datetime import datetime

import os
from sollfahrplanreader.currentdayanalyser import GTFSCurrentDayAnalysis

class SollFahrplanmethodTester(unittest.TestCase):
    
   
    
    def testDateFormtter(self):
        todaysDate='20240514'
        timeToday='22:30:00'
        returedDate=myAnalyser.generateTimeStamp(todaysDate,timeToday)
        print(returedDate)
        actualTimestamp=datetime.strptime("20240514 22:30:00","%Y%m%d %H:%M:%S")
        actualDay=actualTimestamp.day
        actualHour=actualTimestamp.hour
        actualMinute=actualTimestamp.minute
        
        self.assertEqual(actualDay,returedDate.day)
        self.assertEqual(actualHour,returedDate.hour)
        self.assertEqual(actualMinute,returedDate.minute)
    def testServiceParser(self):
        returnedServices=myAnalyser.getTodaysServices()
        numberOfUniqueDays=len(set(list(returnedServices["date"])))
        self.assertEqual(numberOfUniqueDays,1)
        returnedDay=list(returnedServices["date"])[0]
        self.assertEqual(returnedDay,datetime.strftime(datetime.today(),"%Y%m%d"))     
solllTester=SollFahrplanmethodTester()
solllTester.testDateFormtter()

solllTester.testServiceParser()


NameError: name 'GTFSCurrentDayAnalysis' is not defined

In [1]:
from datetime import datetime,timedelta
import os
import pandas as pd
import psycopg2
import pickle
from exceptions.databasealredyfilled import DatabaseAlreadyFilledException
from getrootdirectory import getRootDirectory
from configuration.parameters import ConfigurationParameters


        
class GTFSCurrentDayAnalysis:
    def __init__(self,filePath:str) -> None:
        self.parentDirectory=filePath
        self.configurationParameters=ConfigurationParameters()
    def getTodaysServices(self):
        date_parser = lambda x: pd.to_datetime(x, format='%Y%m%d')
        dayOfWeek=datetime.now().strftime('%A')
        dayOfWeek=dayOfWeek.lower()
        calendar_formatted = pd.read_csv(os.path.join(os.getcwd(),"data","calendar.txt"), parse_dates=['start_date','end_date'], date_parser=date_parser)
        todaysServices=calendar_formatted.loc[((calendar_formatted["start_date"]<date_parser(datetime.today().strftime('%Y%m%d'))) & (calendar_formatted["end_date"]>date_parser(datetime.today().strftime('%Y%m%d'))) & (calendar_formatted[dayOfWeek]==1))]
        calendar_dates=pd.read_csv(os.path.join(os.getcwd(),"data","calendar_dates.txt"),parse_dates=['date'], date_parser=date_parser)
        
        includedsServicesServices=calendar_dates.loc[(calendar_dates["date"]==date_parser(datetime.today().strftime('%Y%m%d'))) & (calendar_dates["exception_type"]==1)]
    

        excludedServvices=calendar_dates.loc[(calendar_dates["date"]==date_parser(datetime.today().strftime('%Y%m%d'))) & (calendar_dates["exception_type"]==2)]
        originalServices=set(list(todaysServices["service_id"]))
        removedServices=set(list(excludedServvices["service_id"]))
        addedServices=set(list(includedsServicesServices["service_id"]))
        activeServices=originalServices.difference(removedServices)
        activeServices=activeServices.union(addedServices)
        todaysServices=pd.DataFrame(columns=["service_id","date"])
        todaysServices["service_id"]=list(activeServices)
        todaysServices["date"]=[datetime.today().strftime("%Y%m%d")]*len(activeServices)
        
        return todaysServices
    
    def getTodaysTrips(self):
        todaysServices=self.getTodaysServices()
        trips=pd.read_csv(os.path.join(self.parentDirectory,"data","trips.txt"))
        todayTrips=pd.merge(trips, todaysServices, on='service_id', how='inner')
        todayUniqueTripds=todayTrips.drop_duplicates(subset=["trip_id"])
        return todayUniqueTripds
    
    def generateTimeStamp(self,day,time:str):
        try:
            timestampStr=day+" "+time
            correct_timestamp=datetime.strptime(timestampStr,"%Y%m%d %H:%M:%S")
        except:
            
            timeparts=time.split(":")
            hours=(timeparts[0])
            minutes=int(timeparts[1])
            correctedHours=correctedHours=int(hours)-24
            current_datetime = datetime.today()

                # Set the time to midnight (00:00:00)
            beginning_of_tomorrow = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)

            correct_timestamp=beginning_of_tomorrow+timedelta(days=1,hours=correctedHours,minutes=minutes)
                
            
        return correct_timestamp
    
    def getDeparturesAndArrivalsToday(self):
        todaysTrips=self.getTodaysTrips()
        dtype = {"date": str}
        stopTimes=pd.read_csv(os.path.join(self.parentDirectory,"data","stop_times.txt"),dtype=dtype)
        first_stop = stopTimes['stop_sequence'] == 1
        last_stop = stopTimes.groupby('trip_id')['stop_sequence'].transform('max') == stopTimes['stop_sequence']
        subset = stopTimes[first_stop | last_stop]
        subset
        
        todaysDepartureArrivals=pd.merge(subset,todaysTrips,on="trip_id",how="inner")
        todaysDepartureArrivals.sort_values(by=['trip_id', 'stop_sequence'])
        todaysDepartureArrivals.to_csv(os.path.join(self.parentDirectory,"data",datetime.now().strftime("%Y-%m-%d")+".txt"))
        
        timestamps=[]
        agencies=[]
        trip_ids=[]
        beginTimes=[]
        endTimes=[]
        agencyFlat=[]
        
        for i in range(len(todaysDepartureArrivals)):
            row=todaysDepartureArrivals.iloc[i]
            day=row["date"]
            currentTripId=row["trip_id"]
            
            
            if(row["stop_sequence"]==1):
                time=row["departure_time"]
                timestamp=self.generateTimeStamp(day,time)
                beginTimes.append(timestamp)
                timestamps.append(timestamp)
                trip_ids.append(currentTripId)
                agency=row["trip_id"].split("-")[0]
                agencyFlat.append(agency)
                
                
                
                        
                
            else:
                    
                
                time=row["arrival_time"]
                timestamp=self.generateTimeStamp(day,time)
                endTimes.append(timestamp)
                timestamps.append(timestamp)
                
                
                
              
                
            agency=row["trip_id"].split(":")[0]
            agencies.append(agency)
            
           
                    
            
        todaysDepartureArrivals["timestamp"]=timestamps
        todaysDepartureArrivals["agency"]=agencies
       
        
        flatDataset=pd.DataFrame(columns=["trip_id","begin_time","end_time","agency"])
        flatDataset["trip_id"]=trip_ids
        flatDataset["begin_time"]=beginTimes
        flatDataset["end_time"]=endTimes
        flatDataset["agency"]=agencyFlat
        flatDataset["number of updates"]=[0]*len(trip_ids)
        flatDataset.to_pickle(os.path.join(self.parentDirectory,"data","flat"+datetime.now().strftime("%Y-%m-%d")+".pkl"))
        #flatDataset.to_pickle("today_flat"+".pkl")
        flatDataset.to_csv(os.path.join(self.parentDirectory,"data","flat"+datetime.now().strftime("%Y-%m-%d")+".txt"))
        
        todaySubset= todaysDepartureArrivals[["trip_id","arrival_time", "departure_time", "stop_id","stop_sequence",
                                        "stop_headsign","route_id", "service_id","trip_headsign","direction_id",
                                        "date", "timestamp", "agency"]]
        todaySubset["minute of day"]=todaySubset['timestamp'].dt.hour * 60 + todaySubset['timestamp'].dt.minute
        todaySubset.to_pickle(os.path.join(self.parentDirectory,"data",datetime.now().strftime("%Y-%m-%d")+".pkl"))
        return flatDataset
    
    def getVolumeSumaryByMinute(self):
        df=self.getDeparturesAndArrivalsToday()[1]
        df['begin_time'] = pd.to_datetime(df['begin_time'])
        df['end_time'] = pd.to_datetime(df['end_time'])

        # Generate a list of unique timestamps within the range of 'begin_time' and 'end_time'
        all_timestamps = pd.date_range(start=df['begin_time'].min().floor('T'), end=df['end_time'].max().ceil('T'), freq='T')

        # Initialize a dictionary to store the count of active trips for each timestamp
        active_trips_count = {}

        # Iterate over each timestamp and count the number of active trips
        for timestamp in all_timestamps:
            active_trips_count[timestamp] = ((df['begin_time'] <= timestamp) & (timestamp <= df['end_time'])).sum()

        # Create a new DataFrame from the dictionary of active trips counts
        active_trips_df = pd.DataFrame.from_dict(active_trips_count, orient='index', columns=['active_trips_count'])

        # Reset the index to have timestamp as a column
        active_trips_df = active_trips_df.reset_index().rename(columns={'index': 'timestamp'})

        # Display the resulting DataFrame
        active_trips_df.to_pickle("minuteSummary"+datetime.now().strftime("%Y-%m-%d")+".pkl")
        return active_trips_df
    
    def prepareSQLStatmentForTrip(self,tripDetailsRow):
        sqlStatement='INSERT INTO schedule_updates(trip_id, begin_time, end_time, agency,number_of_updates) VALUES ({trip_id}, {begin_time},{end_time},{agency},{number_of_updates});'.format(trip_id=tripDetailsRow["trip_id"],begin_time=tripDetailsRow["begin_time"],end_time=tripDetailsRow["end_time"],agency=tripDetailsRow["agency"],number_of_updates=0)
        return sqlStatement
    
    def checkIfDatabaseAlreadyFilled(self):
        rootPath=getRootDirectory()
        try:
            with open(os.path.join(rootPath,"data","flat"+datetime.now().strftime("%Y-%m-%d")+".pkl"), 'rb') as file:
                flat_df = pickle.load(file)
                return True
        except:
            return False
                    
        
        
       
    
    def writeToDatabase(self):
        
        alreadyFilled=self.checkIfDatabaseAlreadyFilled()
        if(alreadyFilled):
            raise DatabaseAlreadyFilledException
        
        todaysTrips=self.getDeparturesAndArrivalsToday()
        databasename=self.configurationParameters.databasename
        conn = psycopg2.connect(
    database=databasename, user='postgres', password='postgres', host='127.0.0.1', port= '5432'
                           )
        cursor = conn.cursor()
        for i in range(len(todaysTrips)):
            currentRow=todaysTrips.iloc[i]
            sqlStatement="INSERT INTO schedule_updates(trip_id, begin_time, end_time, agency,number_of_updates) VALUES (%s, %s, %s, %s, %s)"
            values=(currentRow["trip_id"],currentRow["begin_time"],currentRow["end_time"],currentRow["agency"],0)


            cursor.execute(sqlStatement,values)
            
        conn.commit()
        cursor.close()
        conn.close()   

In [2]:

import os
from getrootdirectory import getRootDirectory
myAnalyser=GTFSCurrentDayAnalysis(getRootDirectory())
todaysTrips=myAnalyser.getDeparturesAndArrivalsToday()
todaysTrips.head()

c:\Users\CHO\Documents\GTFSRTAnalysis\migtfsanalysis


C:\Users\CHO\AppData\Local\Temp\ipykernel_18824\472086160.py:20: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  calendar_formatted = pd.read_csv(os.path.join(os.getcwd(),"data","calendar.txt"), parse_dates=['start_date','end_date'], date_parser=date_parser)
C:\Users\CHO\AppData\Local\Temp\ipykernel_18824\472086160.py:22: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  calendar_dates=pd.read_csv(os.path.join(os.getcwd(),"data","calendar_dates.txt"),parse_dates=['date'], date_parser=date_parser)
C:\Users\CHO\AppData\Local\Temp\ipykernel_18824\472086160.py:69: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  stopTimes=pd.read_csv(os.pa

,trip_id,begin_time,end_time,agency,number of updates
0,AKT:ServiceJourney:100_1_104_281_51_118800_123...,2024-06-17 16:30:00,2024-06-17 17:55:00,AKT:ServiceJourney:100_1_104_281_51_118800_123...,0
1,AKT:ServiceJourney:100_1_112_291_51_126000_131...,2024-06-17 17:30:00,2024-06-17 18:55:00,AKT:ServiceJourney:100_1_112_291_51_126000_131...,0
2,AKT:ServiceJourney:100_1_116_301_69_131400_136...,2024-06-17 18:15:00,2024-06-17 19:45:00,AKT:ServiceJourney:100_1_116_301_69_131400_136...,0
3,AKT:ServiceJourney:100_1_11_71_51_50400_55500_...,2024-06-17 07:00:00,2024-06-17 08:25:00,AKT:ServiceJourney:100_1_11_71_51_50400_55500_...,0
4,AKT:ServiceJourney:100_1_122_311_69_138600_144...,2024-06-17 19:15:00,2024-06-17 20:45:00,AKT:ServiceJourney:100_1_122_311_69_138600_144...,0


In [8]:
from datetime import datetime

d=datetime.today().strftime("%Y%m%d")
l=[d]*3
l

['20240614', '20240614', '20240614']